In [30]:
import os

In [31]:
pwd

'/media/midhu262/01D396A46A88F5C0/concrete_strength_prediction'

In [32]:
os.chdir("/media/midhu262/01D396A46A88F5C0/concrete_strength_prediction/")

In [33]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [34]:
from concrete_strength.constants import *
from concrete_strength.utils.common import read_yaml, create_directories


In [35]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config



In [36]:
import os
import urllib.request as request
import zipfile
from concrete_strength import logger
from concrete_strength.utils.common import get_size

In [37]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [38]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-02-03 20:19:17,313: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-03 20:19:17,316: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-03 20:19:17,319: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-03 20:19:17,322: INFO: common: created directory at: artifacts]
[2024-02-03 20:19:17,331: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-03 20:19:20,221: INFO: 2280763589: artifacts/data_ingestion/concrete_data.zip download! with following info: 
Connection: close
Content-Length: 11347
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "7fc50624ec3f9a6fb55f22e0f78d138cf3120187de3366eba37721402aea0ee1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4386:7A48:BF52A:FFFC7:65BE054F
Accept-Ranges: bytes
Date: Sa